# Notebook with example usage of library

## Chapter 1: Gate on single qubit
Author: Eger Miedema & Jort Leroij<br>
Date: December 17, 2026

In this notebook we will show you how to use our library

First we import the necisary libraries

In [154]:
from pathlib import Path
import sys
import numpy as np

### Importing the local library

Since this notebook is run directly from the repository, we first ensure that
the project root is available on the Python path so that the `nqubitsim` package
can be imported correctly.



In [155]:
project_root = Path.cwd().resolve().parent
sys.path.insert(0, str(project_root))

from src.nqubitsim.simulator import QuantumSimulator
from src.nqubitsim import gates


### Step 2: Creating a quantum simulator

We create a simulator with a single qubit.
By default, the qubit starts in the state |0⟩.


In [156]:
sim = QuantumSimulator(num_qubits=1)

### Step 3: Inspecting the initial state

Before applying any gates, we look at the probability of measuring |0⟩ or |1⟩.


In [157]:
probs = sim.state.probabilities([0])
print("Probabilities for |0> and |1>:", probs)

Probabilities for |0> and |1>: [1. 0.]


### Step 4: Applying a quantum gate

We apply a Hadamard (H) gate to the qubit.
This puts the qubit into a superposition of |0⟩ and |1⟩.


In [158]:
sim.apply_gate(gates.H, target=0)

probs = sim.state.probabilities([0])
print("Probabilities after H gate:", probs)

Probabilities after H gate: [0.5 0.5]


### Step 5: Measuring the qubit

We now perform a measurement.
The measurement result is classical and collapses the quantum state.


In [159]:
outcome, post_state = sim.measure([0]) # Measure qubit 0

print("Measured outcome:", outcome)
print("Classical register:", sim.classical_register)
print("Post-measurement density matrix:")
print(post_state.get_density())

Measured outcome: 1
Classical register: [1]
Post-measurement density matrix:
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


### Step 6: Resetting the simulator

After a measurement, the simulator can be reset to start a new experiment.


In [160]:
sim.reset()

probs = sim.state.probabilities([0])
print("Probabilities after reset:", probs)

Probabilities after reset: [1. 0.]


## Chapter 2: Two Qubits and Entanglement
In this chapter we extend the simulator to two qubits.
We apply multiple gates and observe how measuring one qubit affects the other.
This introduces the concept of entanglement.

### Step 1: Creating a two-qubit simulator

We now create a simulator with two qubits.
The initial state of the system is |00⟩.

In [161]:
sim = QuantumSimulator(num_qubits=2)

### Step 2: Inspecting the initial state

We look at the probabilities of measuring the four possible basis states:
|00⟩, |01⟩, |10⟩, and |11⟩.

In [162]:
probs = sim.state.probabilities([0, 1])
print("Probabilities for |00>, |01>, |10>, |11>:", probs)

Probabilities for |00>, |01>, |10>, |11>: [1. 0. 0. 0.]


### Step 3: Creating superposition on the first qubit
We apply a Hadamard gate to the first qubit.
This creates a superposition while the second qubit remains unchanged.

In [163]:
sim.apply_gate(gates.H, target=0)

probs = sim.state.probabilities([0, 1])
print("Probabilities after H on qubit 0:", probs)

Probabilities after H on qubit 0: [0.5 0.  0.5 0. ]


### Step 4: Applying a controlled gate
We apply a CNOT gate with:
- control qubit: 0
- target qubit: 1

This operation entangles the two qubits.

In [164]:
sim.apply_controlled_gate(gates.CNOT, control=0, target=1)

probs = sim.state.probabilities([0, 1])
print("Probabilities after CNOT:", probs)

Probabilities after CNOT: [0.5 0.  0.  0.5]


### Step 5: Interpreting the result

The system is now in the entangled state:

(|00⟩ + |11⟩) / √2

Only |00⟩ and |11⟩ have non-zero probability.
The qubits can no longer be described independently.

### Step 6: Measuring the qubits

We now measure both qubits.
The measurement result is classical and collapses the quantum state.

In [165]:
outcome, post_state = sim.measure([0, 1])

print("Measured outcome (integer):", outcome)
print("Classical register:", sim.classical_register)
print("Post-measurement density matrix:")
print(post_state.get_density())

Measured outcome (integer): 3
Classical register: [3]
Post-measurement density matrix:
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


### Step 7: Repeating the experiment

To see the probabilistic nature of measurement, we repeat the experiment
multiple times.

In [166]:
sim.reset()

shots = 10
counts = {0: 0, 1: 0, 2: 0, 3: 0}

for _ in range(shots):
    sim.apply_gate(gates.H, target=0)
    sim.apply_controlled_gate(gates.CNOT, control=0, target=1)
    outcome, _ = sim.measure([0, 1])
    counts[outcome] += 1
    sim.reset()

print(f"Measurement results over {shots} shots:")
print("0 = |00>, 3 = |11>")
print(counts)

Measurement results over 10 shots:
0 = |00>, 3 = |11>
{0: 3, 1: 0, 2: 0, 3: 7}
